In [ ]:
import os
from pathlib import Path

import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import panel as pn
import proplot as pplt
import seaborn as sns
import xarray as xr
from apertools import lowess, plotting, utils
from numpy.fft import fft, fftfreq, fftshift, ifft
from scipy import signal, stats
from troposim import igram_sim, turbulence

pn.extension()
# pplt.rc.update({"subplots.share": False, "subplots.span": False})


sns.set_style(style="white")
# plotting.set_style(size=16)


RNG = np.random.default_rng()

%matplotlib inline
# %matplotlib widget

%load_ext autoreload
%autoreload 2

In [ ]:
# sns.set_style?

# Chapter: Improved Robust Long-term Time Series Methods

1. Limitations of stacking to long time series
2. Robust Time Series Methods
    1. Regularization (Supplement from GRL)
    1. LPF by triangle filiter 
    1. LOWESS smoothing
3. Synthetic Example
    1. frequency response of triangle
    1. response of LOWESS
4. 7 Year Time Series for the Permian Basin
    1. Comparison to GPS
    2. Anthropogenic Caused Deformation Patterns

In [ ]:
r = 100
b = 2.7
N = 10
shape = (100, 100)
ny, nx = shape

igm_g = igram_sim.IgramMaker(
    num_days=N,
    shape=shape,
    distribution="normal",
    resolution=r,
    p0_default=100,
    to_cm=True,
)
igm_u = igram_sim.IgramMaker(
    num_days=N,
    shape=shape,
    distribution="uniform",
    resolution=r,
    p0_default=100,
    to_cm=True,
)

sar_stack_u, sar_date_list = igm_u.make_sar_stack(beta=b)
sar_stack_g, _ = igm_g.make_sar_stack(beta=b)

In [ ]:
sar_stack_g.max(), sar_stack_g.min()

In [ ]:
plt.figure()
plt.imshow(sar_stack_u[0])
plt.colorbar()

In [ ]:
s = sar_stack_u[sar_stack_u.max(axis=(1, 2)).argmax()]

plt.figure()
plt.imshow(s)
plt.colorbar()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3), sharex=True, sharey=True, squeeze=False)
axes = axes.ravel()

# axes[0].hist(sar_stack1.ravel())
# axes[1].hist(sar_stack2.ravel())
ax = axes[0]
for s in sar_stack_g:
    sns.kdeplot(x=s.ravel(), ax=ax)
ax.set_title("Gaussian")

ax = axes[1]
for s in sar_stack_u:
    sns.kdeplot(x=s.ravel(), ax=ax)
ax.set_title("Uniform")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), sharex=True, sharey=True)

sns.kdeplot(x=sar_stack_g.ravel(), ax=ax, label="Gaussian")
sns.kdeplot(x=sar_stack_u.ravel(), ax=ax, label="Uniform")
ax.legend()

# Add defo 

In [ ]:
p0_rv = "gamma"
p0_params = {"scale": 50, "a": 2, "loc": 2}

plt.figure()
xx = np.linspace(0, 250)

plt.plot(xx, stats.gamma(**p0_params).pdf(xx))
plt.xlabel("p0")

In [ ]:
defo_rates = [2, -2]
# num_days = 80 # 2.6 years
num_days = 91  # 3 years
r = 200  # resolution, m
# b = 2.7

shape = (300, 300)

igm = igram_sim.IgramMaker(
    num_days=num_days, shape=shape, resolution=r, to_cm=True, defo_rates=defo_rates
)

In [ ]:
# Draw the daily powers from an exponential

seed = 1
beta_arr_slopes = stats.norm.rvs(
    loc=-2.6, scale=0.1, size=(num_days,), random_state=seed
)
beta_arr = np.vstack((-9.7 * np.ones(num_days), beta_arr_slopes)).T

sar_stack, sar_date_list = igm.make_sar_stack(
    beta=beta_arr, p0_params=p0_params, p0_rv=p0_rv, seed=seed
)
dd = igm.make_defo_stack(sigma=15)

In [ ]:
sns.histplot(np.max(np.abs(sar_stack), axis=(1, 2)), bins=20)

In [ ]:
plt.figure()
plt.imshow(igm.sar_stack[4])
plt.colorbar()

In [ ]:
plt.figure()
plt.imshow(dd[4])
plt.colorbar()

In [ ]:
dates = pd.to_datetime(igm.sar_date_list)
dd_xr = utils.stack_to_xr(dd, z_coords=dates, dims=("date", "y", "x"))
noise_xr = utils.stack_to_xr(igm.sar_stack, z_coords=dates, dims=("date", "y", "x"))

In [ ]:
# dd_xr
# noise_xr

In [ ]:
plotting.hvplot_stack(dd_xr, x="x", y="y")

In [ ]:
plotting.hvplot_stack(noise_xr, x="x", y="y")

# Time series of center of deformation

In [ ]:
noise = igm.sar_stack.copy()
noise = noise - noise[:, 5, 5][:, None, None]
# Make the day1 noise non-trivial
# day1max = np.abs(noise[0]).max()

# noisemids = np.abs(noise[:, ny//2, nx//2])
noisemids = noise[:, ny // 2, nx // 2]
# mid_idx = np.argmax(noisemids)
mid_idx = np.argsort(noisemids)[-3]  # get 3rd largest, not max
img1, img2 = noise[0], noise[mid_idx]
noise[0] = img2
noise[mid_idx] = img1

# newmax = 2
# noise[0] = (noise[0] / day1max) * newmax
# save for later
day1noise = noise[0].copy()
# Subtract the day 1 from the rest:
noise = noise - noise[0][None, :, :]

defo = igm.defo_stack.copy()
defo_plus_noise = noise + defo
dn_xr = utils.stack_to_xr(defo_plus_noise, z_coords=dates, dims=("date", "y", "x"))

In [ ]:
day1noise.max(), day1noise.min()

In [ ]:
nmax = np.abs(day1noise).max()
plt.imshow(day1noise, cmap="seismic_wide_y_r", vmax=nmax, vmin=-nmax)
plt.colorbar()

In [ ]:
plotting.hvplot_stack(dn_xr, x="x", y="y")

In [ ]:
dn_xr.std(dim=("date")).shape

In [ ]:
plt.figure()
dn_xr.std(dim=("date")).plot.imshow()
plt.figure()
noise_xr.std(dim=("date")).plot.imshow()

In [ ]:
ts_noisy = dn_xr.sel(x=nx // 2, y=ny // 2).copy()
ts_truth = dd_xr.sel(x=nx // 2, y=ny // 2).copy()

sigma = 0.7
ts_white_noise = ts_truth.copy() + RNG.normal(scale=sigma, size=len(ts_noisy))
ts_white_noise -= ts_white_noise[0]

In [ ]:
# stats.kstest?

In [ ]:
(
    stats.kstest(ts_noisy.data - ts_truth.data, stats.norm.cdf),
    stats.kstest(stats.norm.rvs(size=100), stats.norm.cdf),
    stats.kstest(ts_white_noise.data - ts_truth.data, stats.norm.cdf),
)

In [ ]:
fig, ax = plt.subplots()
ts_white_noise.plot(label="white noise", ax=ax)
ts_noisy.plot(label="turbulence", ax=ax)
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
sns.kdeplot(stats.norm.rvs(size=len(ts_noisy), scale=sigma), label="norm.rvs")
sns.kdeplot(ts_white_noise.data - ts_truth.data, label="white")
sns.kdeplot(ts_noisy.data - ts_truth.data, label="turb")
ax.legend()

# LOWESS Algorithm Demonstration

## LOWESS windows only

In [ ]:
fig, ax = pplt.subplots(refwidth=5, refheight=2, grid=True)
xx = np.linspace(0, 1, 100)
idx = 20

for frac, fstr in zip([1 / 6, 1 / 3, 1 / 2], ["1/6", "1/3", "1/2"]):
    w = lowess.demo_window(xx, frac=frac)
    ax.plot(xx, w[:, idx], label=f"$\gamma =${fstr}", lw=3)

# ax.legend(loc='upper left')
ax.legend(loc="upper right", ncols=1)

ax.set_ylabel("$\mathbf{w}(t_i)$")

# ax.set_ylim((None, .1))
ax.set_xticks([idx / 100], ["$t_i$"])
ax.set_xlabel("$\mathbf{t}$")

In [ ]:
# f = 1/2
# x = xx

# n = len(x)
# r = int(np.ceil(f * n))
# h = [np.sort(np.abs(x - x[i]))[r] for i in range(n)]
# w = np.clip(np.abs((x[:, None] - x[None, :]) / h), 0.0, 1.0)
# w = (1 - w ** 3) ** 3
# plt.figure(); plt.plot(w[:, 40])

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure1-window.pdf")

## LOWESS windows, time series example

In [ ]:
dn_xr_noisy = dn_xr.copy()

# Make something near the demo index noisy
idx = 20

dn_xr_noisy[idx + 7] += dn_xr_noisy[idx + 7] + 5
dn_xr_noisy[idx + 4] += -dn_xr_noisy[idx + 4] + 6.5
# noiseidx = 59
noiseidx = 51
dn_xr_noisy[noiseidx] += -dn_xr_noisy[noiseidx] - 3.7
dn_xr_noisy[noiseidx + 1] += -dn_xr_noisy[noiseidx + 1] - 2.9

ts_noisy = dn_xr_noisy.sel(x=nx // 2, y=ny // 2).copy()
# ts_noisy[idx + 7] = 5
# ts_noisy[idx + 4] = 6.5
# ts_noisy[59] = -3.7
# ts_noisy[60] = -2.9

x = lowess.date2num(dn_xr.date.values)
x = (x - x[0]) / 365.25
# x

In [ ]:
plt.figure()
plt.plot(dn_xr_noisy[:, nx // 2, ny // 2])
plt.plot(dn_xr[:, nx // 2, ny // 2])

In [ ]:
pplt.rc["cycle"] = "colorblind"
pplt.rc["grid.alpha"] = 0.4

In [ ]:
fig, ax = pplt.subplots(refwidth=5, refheight=2, grid=True)
ax2 = ax.twinx()

# lon, lat = gps.station_lonlat("TXKM")
# ax2.scatter(igm.sar_date_list, ts_noisy.data, label=r"$\phi$", marker='.', lw=1, color="gold", s=100)
# Using the spiked
ax2.scatter(
    igm.sar_date_list, y, label=r"$\phi$", marker=".", lw=1, color="gold", s=100
)


years_smooth = [0.5, 1, 1.5]
for year in years_smooth:
    w = lowess.demo_window(x, min_x_weighted=year)
    ax.plot(igm.sar_date_list, w[:, idx], label=f"{year} year", lw=3)

# ax.legend(loc='upper left')
ax.legend(loc="upper right", ncols=1)
# ax2.legend()
# ax.set_title("Initial window weight for each LOWESS iteration")
ax2.set_title("")
ax2.set_ylabel(f"$\phi$ [cm]")

# ax2.set_ylim((-7, 7))
ax.set_ylabel("$\mathbf{w}(t_i)$")

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure1-window-ts.pdf")

In [ ]:
# fig, ax = plt.subplots()
# ax2 = ax.twinx()

# # lon, lat = gps.station_lonlat("TXKM")
# ts_noisy.plot(ax=ax2, label="insar", marker='.', lw=1, color='gold')


# years_smooth = [0.5, 1, 1.5]
# for year in years_smooth:
#     w = lowess.demo_window(x, min_x_weighted=year)
#     ax.plot(igm.sar_date_list, w[50], label=f"{year} year", lw=3)

# ax.legend(loc='upper left')
# # ax2.legend()
# ax.set_title("Initial window weight for each LOWESS iteration")
# ax2.set_title("")
# ax2.set_ylabel("InSAR [cm]")

# # ax2.set_ylim((-7, 7))
# ax.set_ylabel("$\mathbf{w}(t_i)$")

## Time series only plot, plus image of deformation

In [ ]:
ts_arr = [ts_truth, ts_noisy]
# labels = ['Noisy', 'Truth']
labels = ["$\mathbf{d}$", "$\mathbf{\phi}$"]
# colors = ["C1", "C0"]
colors = ["C1", "gray"]
# colors = ["C0", "gray"]

# cmap_img = 'RdBu_r'
cmap_img = "seismic_wide_y"


fig, axes = pplt.subplots([[1, 2], [3, 3]], abc="(a)")  # , figsize=(6, 3))


ax.set_ylabel("[cm]")
# ax.set_title("Triangle-window smoothing")
# ax.grid()

ax.legend()
defo_img = dd_xr[len(dd_xr) // 2]
img_date = dd_xr.indexes["date"][len(dd_xr) // 2]
defo_noise_img = dn_xr[len(dd_xr) // 2]
vm = np.abs(defo_noise_img).max()


ax = axes[0]
# axim = ax.imshow(noise_xr[len(dd_xr)//2], cmap=cmap_img, vmax=vm, vmin=-vm)
axim = ax.imshow(defo_img, cmap=cmap_img, vmax=vm, vmin=-vm)
ax.colorbar(axim, loc="r", label="[cm]")
ax.format(grid=False, title=f"{labels[0]}: {str(img_date.date())}")
ax.plot(nx // 2, ny // 2, "X", color="black")


ax = axes[1]
axim = ax.imshow(defo_noise_img, cmap=cmap_img, vmax=vm, vmin=-vm)
ax.colorbar(axim, loc="r", label="[cm]")
ax.format(grid=False, title=f"{labels[1]}: {str(img_date.date())}")
ax.plot(nx // 2, ny // 2, "X", color="black")

ax = axes[2]

dts = ts_noisy.date.values
# for tt, label, color in zip(ts_arr, labels, colors):
ax.plot(dts, ts_arr[0], lw=3, label=labels[0], color=colors[0])
ax.plot(dts, ts_arr[1], lw=1.5, label=labels[1], marker=".", color=colors[1])
ax.set_ylabel("[cm]")
ax.legend(loc="ur")

ylim = ax.get_ylim()  # Keep the same limits, but make a vert line for whole span
ax.vlines(img_date, ylim[0], ylim[1], linestyle="--", color="gray", alpha=0.5)
ax.set_ylim(ylim)

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure2-demo-data.pdf")

### Color tests

In [ ]:
# First Iteration
from matplotlib.colors import LinearSegmentedColormap, to_rgb, to_rgba


def get_alpha_by_value(alpha_arr, color="C1"):
    r, g, b = to_rgb(color)
    # r, g, b, _ = to_rgba(color)
    return [(r, g, b, alpha) for alpha in alpha_arr]


get_alpha_by_value([0.5, 0.2, 1])
# fig, axs = pplt.show_cycles(rasterized=True)
# pplt.rc['cycle'] = 'tab10'

In [ ]:
# LinearSegmentedColormap.from_list('', [to_rgba("C2"), to_rgba("grey")], N)
cmap = LinearSegmentedColormap.from_list("", [to_rgba("grey"), to_rgba("gold")], N)
cmap

## LOWESS window color shading

In [ ]:
# idx = 20
# w = lowess.demo_window(x, min_x_weighted=1.0)
# window = w[:, idx]

# # color_arr = get_alpha_by_value(window)
# # map from gray at 0 to green at max
# N = 256
# # cmap = LinearSegmentedColormap.from_list('', [to_rgba("C6"), to_rgba("C8")], N)
# colors = cmap(window)

# # fig, ax = pplt.subplots(figsize=(8, 5))
# fig, ax = pplt.subplots(refwidth=5, refheight=2)

# ax2 = ax.twinx()

# # lon, lat = gps.station_lonlat("TXKM")
# aa = ax2.scatter(igm.sar_date_list, ts_noisy.data, label="insar", marker='.', lw=1, color=colors, s=100)


# ax.plot(igm.sar_date_list, window, lw=3)

# # ax.legend(loc='upper left')
# # ax2.legend()
# # ax.set_title("Initial window weight for initial LOWESS iteration")
# ax2.set_title("")
# ax2.set_ylabel("InSAR [cm]")

# # ax2.set_ylim((-7, 7))
# ax.set_ylabel("$\mathbf{w}(t_i)$")

# # cmap

## LOWESS First iteration slope fit

In [ ]:
idx = 20
w = lowess.demo_window(x, min_x_weighted=1.0)
beta = lowess.demo_fit(x, ts_noisy.data, w, idx)
x_slice = x[idx - 10 : idx + 10]
yest = beta[0] + beta[1] * x_slice
# yest

In [ ]:
from matplotlib.colors import LinearSegmentedColormap, to_rgba

w = lowess.demo_window(x, min_x_weighted=1.0)
window = w[:, idx]

# color_arr = get_alpha_by_value(window)
# map from gray at 0 to green at max
N = 256
# cmap = LinearSegmentedColormap.from_list('', [to_rgba("C6"), to_rgba("C8")], N)
colors = cmap(window)

# fig, ax = plt.subplots(figsize=(8, 5))
fig, ax = pplt.subplots(refwidth=5, refheight=2)
ax2 = ax.twinx()

fit_color = "C1"
# lon, lat = gps.station_lonlat("TXKM")
ax2.scatter(
    igm.sar_date_list,
    ts_noisy.data,
    label=r"$\phi$",
    marker=".",
    lw=1,
    color=colors,
    s=100,
)
ax2.plot(igm.sar_date_list[idx - 10 : idx + 10], yest, lw=6, color=fit_color)

ax.plot(igm.sar_date_list, window, lw=3)

# ax.legend(loc='upper left')
# ax2.legend()
ax.set_title("Initial fit")
ax2.set_title("")
ax2.set_ylabel("InSAR [cm]")

# ax2.set_ylim((-7, 7))
ax.set_ylabel("$\mathbf{w}(t_i)$")

In [ ]:
resid = lowess.demo_residual(x, ts_noisy.data, w, idx)
beta_2 = lowess.demo_fit(x, ts_noisy.data, w, idx, delta=resid)
yest_2 = beta_2[0] + beta_2[1] * x_slice
yest_2

In [ ]:
from matplotlib.colors import LinearSegmentedColormap, to_rgba

w = lowess.demo_window(x, min_x_weighted=1.0)
window = w[:, idx]
window2 = resid * window

# color_arr = get_alpha_by_value(window)
# map from gray at 0 to green at max
# N = 256
# cmap = LinearSegmentedColormap.from_list('', [to_rgba("C6"), to_rgba("C8")], N)
colors2 = cmap(window2)

# fig, ax = plt.subplots(figsize=(8, 5))
fig, ax = pplt.subplots(refwidth=5, refheight=2)

ax2 = ax.twinx()

fit_color = "C1"
fit_color2 = "C2"
# lon, lat = gps.station_lonlat("TXKM")
ax2.scatter(
    igm.sar_date_list,
    ts_noisy.data,
    label="$\phi$",
    marker=".",
    lw=1,
    color=colors2,
    s=100,
)
ax2.plot(igm.sar_date_list[idx - 10 : idx + 10], yest, lw=5, color=fit_color, alpha=0.5)
ax2.plot(igm.sar_date_list[idx - 10 : idx + 10], yest_2, lw=6, color=fit_color2)

# ax.plot(igm.sar_date_list, window, lw=3)
ax.plot(igm.sar_date_list, window2, lw=3)

# ax.legend(loc='upper left')
# ax2.legend()
# ax.set_title("Second iteration fit")
ax2.set_title("")
ax2.set_ylabel("\phi [cm]")

# ax2.set_ylim((-7, 7))
ax.set_ylabel("$\mathbf{w}(t_i)$")

## Combined first 2 iterations

In [ ]:
# Recalculate the 2 estimates
idx = 20

w = lowess.demo_window(x, min_x_weighted=1.0)
beta = lowess.demo_fit(x, ts_noisy.data, w, idx)
x_slice = x[idx - 10 : idx + 10]
yest = beta[0] + beta[1] * x_slice


resid = lowess.demo_residual(x, ts_noisy.data, w, idx)
beta_2 = lowess.demo_fit(x, ts_noisy.data, w, idx, delta=resid)
yest_2 = beta_2[0] + beta_2[1] * x_slice
yest_2

In [ ]:
y = ts_noisy.data
y_iter1 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=1)
y_iter2 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=2)


w = lowess.demo_window(x, min_x_weighted=1.0)
window = w[:, idx]


N = 256
colors = cmap(window)

# fig, axes = pplt.subplots(refwidth=4, refheight=1.2, nrows=3, abc="(a)")
fig, axes = pplt.subplots(ncols=2, nrows=2, abc="(a)", sharey=False)
# fig, ax = plt.subplots(figsize=(8, 5))

ax1 = axes[0]
ax2 = ax1.twinx()

fit_color = "C1"
ax1.scatter(
    igm.sar_date_list,
    ts_noisy.data,
    label=r"$\phi$",
    marker=".",
    lw=1,
    color=colors,
    s=100,
)
ax1.plot(igm.sar_date_list[idx - 10 : idx + 10], yest, lw=6, color=fit_color)
ax1.set_ylabel("$\phi$ [cm]")

ax2.plot(igm.sar_date_list, window, lw=3, color=fit_color)
# ax2.set_ylim((-7, 7))
ax2.set_ylabel("$\mathbf{w}(t_i)$")

######################################

ax = axes[1]

fit_color = "C1"
ax.scatter(
    igm.sar_date_list, ts_noisy.data, marker=".", lw=1, color="gray", s=100
)  # , label=r"$\phi$")
ax.plot(igm.sar_date_list, y_iter1, lw=4, color=fit_color, label="1 iters")


ax.set_ylabel("$\phi$ [cm]")


###############################
window2 = resid * window
colors2 = cmap(window2)

ax1 = axes[2]
ax2 = ax1.twinx()

fit_color = "C1"
fit_color2 = "C2"

alpha = 0.7
ax1.scatter(
    igm.sar_date_list,
    ts_noisy.data,
    label="$\phi$",
    marker=".",
    lw=1,
    color=colors2,
    s=100,
)
ax1.plot(
    igm.sar_date_list[idx - 10 : idx + 10], yest, lw=5, color=fit_color, alpha=alpha
)
ax1.plot(igm.sar_date_list[idx - 10 : idx + 10], yest_2, lw=6, color=fit_color2)
ax1.set_ylabel("$\phi$ [cm]")

ax2.plot(igm.sar_date_list, window2, lw=3, color=fit_color2)
# ax2.set_ylim((-7, 7))
ax2.set_ylabel("$\mathbf{w}(t_i)$")


######################################
ax = axes[3]

fit_color = "C1"
ax.scatter(
    igm.sar_date_list, ts_noisy.data, marker=".", lw=1, color="gray", s=100
)  # , label=r"$\phi$")
ax.plot(igm.sar_date_list, y_iter1, lw=4, color=fit_color, alpha=alpha, label="1 iters")
ax.plot(igm.sar_date_list, y_iter2, lw=3, color=fit_color2, label="2 iters")


ax.set_title("")
ax.set_ylabel("$\phi$ [cm]")
ax.legend(loc="lower left", ncols=1)

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure3-fits.pdf")

In [ ]:
# from matplotlib.colors import LinearSegmentedColormap, to_rgba

# w = lowess.demo_window(x, min_x_weighted=1.0)
# window = w[:, idx]

# # color_arr = get_alpha_by_value(window)
# # map from gray at 0 to green at max
# N = 256
# cmap = LinearSegmentedColormap.from_list('', [to_rgba("C6"), to_rgba("C8")], N)
# colors = cmap(window)

# fig, ax = plt.subplots(figsize=(8, 5))
# ax2 = ax.twinx()

# # lon, lat = gps.station_lonlat("TXKM")
# ax2.scatter(igm.sar_date_list, ts_noisy.data, label="insar", marker='.', lw=1, color=colors, s=100)
# ax2.plot(igm.sar_date_list[idx-10:idx+10], yest, lw=5, color='gray')
# ax2.plot(igm.sar_date_list[idx-10:idx+10], yest_2, lw=6)

# ax.plot(igm.sar_date_list, window, lw=3)

# ax.legend(loc='upper left')
# # ax2.legend()
# ax.set_title("Second iteration fit")
# ax2.set_title("")
# ax2.set_ylabel("InSAR [cm]")

# # ax2.set_ylim((-7, 7))
# ax.set_ylabel("$\mathbf{w}(t_i)$")

In [ ]:
y_iter2_zeroed = y_iter2 - y_iter2[0]
ts_arr = [ts_truth, ts_noisy, y_iter2_zeroed]
# labels = ['Noisy', 'Truth']
labels = ["$\mathbf{d}$", "$\mathbf{\phi}$", "$\mathbf{\hat{\phi}}$"]
colors = ["C1", "C0", "C2"]
# [[1, 2], [3, 3]],
fig, axes = pplt.subplots(abc="(a)", figsize=(6, 3))

ax = axes[0]
dts = ts_noisy.date.values
# for tt, label, color in zip(ts_arr, labels, colors):
ax.plot(dts, ts_truth, lw=4, label=labels[0], color=colors[0])
ax.plot(dts, ts_noisy, lw=2, label=labels[1], marker=".", color=colors[1])
ax.plot(dts, y_iter2_zeroed, lw=4, label=labels[2], color=colors[2])

ax.set_ylabel("[cm]")
ax.legend(loc="ur")

## Plot fits from first 2 iterations

In [ ]:
y = ts_noisy.data
y_iter1 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=1)
y_iter2 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=2)


fig, axes = pplt.subplots(refwidth=3.5, refheight=1.5, nrows=1, abc="(a)")


ax = axes[0]

fit_color = "C1"
ax.scatter(
    igm.sar_date_list, ts_noisy.data, marker=".", lw=1, color="gray", s=100
)  # , label=r"$\phi$")
ax.plot(igm.sar_date_list, y_iter1, lw=4, color=fit_color, label="1 iters")
ax.plot(igm.sar_date_list, y_iter2, lw=3, color=fit_color2, label="2 iters")


ax.set_title("")
ax.set_ylabel("$\phi$ [cm]")
ax.legend(loc="lower left")

## Changes for each iteration

In [ ]:
plt.figure()
yy = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=1)
for nn in range(2, 5):
    yprev = yy
    yy = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=nn)
    plt.plot(yy - yprev, label=f"{nn} - {nn - 1}")
plt.legend()

## Compare Triangle to LOWESS

### Do the whole image

In [ ]:
# dn_xr

In [ ]:
%%time
frac = 0.4
x2 = lowess.date2num(dn_xr.date.values)
# dn_xr_lowess = lowess.lowess_xr(dn_xr, frac=1/3) #, min_days_weighted=365.0)
dn_lowess = lowess.lowess_stack(
    # dn_xr.data, x2, frac=1 / 3
    dn_xr_noisy.data,
    x2,
    frac=frac,
)  # , min_days_weighted=365.0)

In [ ]:
dn_lowess = dn_lowess - dn_lowess[0][None, :, :]
dn_xr_lowess = dn_xr.copy()
dn_xr_lowess.data = dn_lowess
dn_xr_lowess.shape

In [ ]:
# dn_xr_lowess[:, nx//2, ny//2].plot()
# fig, axes = dn_xr_lowess[len(x)//2].plot.imshow()

In [ ]:
from scipy import ndimage

mode = "reflect"
tri_win = signal.windows.triang(int(frac * len(x2)))

dn_tri = ndimage.convolve1d(dn_xr_noisy.data, tri_win, axis=0, mode=mode) / sum(tri_win)
dn_tri = dn_tri - dn_tri[0][None, :, :]
dn_xr_tri = dn_xr.copy()
dn_xr_tri.data = dn_tri
# dn_xr_tri

In [ ]:
fig, ax = pplt.subplots()

dn_xr_lowess[:, nx // 2, ny // 2].plot(ax=ax)
dn_xr_tri[:, nx // 2, ny // 2].plot(ax=ax)

In [ ]:
fig, ax = pplt.subplots()

dn_xr_lowess[:, nx // 2, ny // 2].plot(ax=ax)
dn_xr_tri[:, nx // 2, ny // 2].plot(ax=ax)

In [ ]:
iidx = len(x) // 2
iidx = 62

zoom = (slice(120, 180), slice(120, 180))
# zoom = (slice(None), slice(None))

plotting.plot_img_diff(
    arrays=(
        dn_xr_tri[iidx][zoom],
        dn_xr_lowess[iidx][zoom],
        dd_xr[iidx][zoom],
    ),
    show_diff=False,
    titles=["Triangle", "LOWESS", "Truth (no noise)"],
    vm=3,
    cbar_label="[cm]",
    # cmap="RdBu_r",
    cmap="seismic_wide_y",
    axis_off=True,
)

In [ ]:
iidx = len(x) // 2
iidx = 62

zoom = (slice(120, 180), slice(120, 180))
# zoom = (slice(None), slice(None))

plotting.plot_img_diff(
    arrays=(
        dn_xr_tri[iidx][zoom],
        dn_xr_lowess[iidx][zoom],
        dd_xr[iidx][zoom],
    ),
    show_diff=False,
    titles=["Triangle", "LOWESS", "Truth (no noise)"],
    vm=3,
    cbar_label="[cm]",
    # cmap="RdBu_r",
    cmap="seismic_wide_y_r",
    axis_off=True,
)

In [ ]:
def rms(x):
    return np.sqrt((x ** 2).mean())

In [ ]:
tri_error = dn_xr_tri[iidx] - dd_xr[iidx]
lowess_error = dn_xr_lowess[iidx] - dd_xr[iidx]

# rms(dn_xr_tri[iidx] - dd_xr[iidx]), rms(dn_xr_lowess[iidx] - dd_xr[iidx]),
np.abs(tri_error).max(), np.abs(lowess_error).max(),

In [ ]:
fig, axes = pplt.subplots(ncols=1)
ax = axes
ax.hist(tri_error.data.ravel(), bins=50, label="tri")

ax.hist(lowess_error.data.ravel(), bins=50, label="lowess", alpha=0.6)
ax.legend()
ax.set_xlim((-1.5, 1.5))

In [ ]:
iidx = len(x) // 2
iidx = 62
plotting.plot_img_diff(
    arrays=(tri_error, lowess_error),
    # show_diff=False,
    titles=["Triangle error", "LOWESS error"],
    vm=3,
)

In [ ]:
# # signal.windows.gaussian
# tri_win = signal.windows.triang(int(1 / 3 * len(x)))
# ts_tri = signal.convolve(ts_noisy, tri_win, mode="same") / sum(tri_win)
# ts_tri -= ts_tri[0]


# fig, ax = plt.subplots()

# ax.plot(ts_noisy.date, ts_tri, lw=4, label="", marker=".")


# ax.set_title("Triangle-window smoothing")
# ax.grid()
# ax.legend()

### attempt the mean thing for removing day1

In [ ]:
diffs = dn_xr.data - dn_xr.data[0][None, :, :]
diffs.shape
fig, axes = pplt.subplots(ncols=3)

day1noise_est = -1 * diffs.mean(axis=0)
axes.format(title=["Est", "Truth", "Left - middle"])

ax = axes[0]
axim = ax.imshow(day1noise_est)
ax.colorbar(axim, loc="r")

ax = axes[1]
axim = ax.imshow(day1noise)
ax.colorbar(axim, loc="r")


ax = axes[2]
axim = ax.imshow(day1noise_est - day1noise)
ax.colorbar(axim, loc="r")

s = (150, 150)
print(day1noise_est[s], day1noise[s])

### Plot

In [ ]:
for f in [1 / 6, 1 / 4, 1 / 3, 0.4]:
    print(lowess.lowess_pixel(y, x, frac=f, n_iter=2)[0])

In [ ]:
plt.plot(ts_truth - ts_tri)
plt.plot(y_iter2_zeroed - ts_tri)
plt.grid()

In [ ]:
# fig, axes = pplt.subplots(figsize=(3, 5))
fig, axes = pplt.subplots(
    nrows=2, ncols=2, sharex=False, sharey=False, share=False, abc="(a)"
)


y = ts_noisy.data
# frac = 1/3
# frac = 0.4
# y_iter1 = lowess.lowess_pixel(y, x, frac=frac, n_iter=1)
y_iter2 = lowess.lowess_pixel(y, x, frac=frac, n_iter=2)
y_iter2_zeroed = y_iter2 - y_iter2[0]

tri_win = signal.windows.triang(int(frac * len(x)))
# Add the estimate of the day1 noise
ts_noisy_remove_est = ts_noisy.copy() + day1noise_est[s]
ts_noisy_remove_est[0] = 0

# To skip the "remove the average ifg phase"
# ts_noisy_remove_est = ts_noisy.copy()

ts_tri = signal.convolve(ts_noisy_remove_est, tri_win, mode="same") / sum(tri_win)
ts_tri_zeroed = ts_tri - ts_tri[0]


ts_arr = [ts_truth, ts_noisy, y_iter2_zeroed, ts_tri]
# labels = ['Noisy', 'Truth']
labels = ["$\mathbf{d}$", "$\mathbf{\phi}$", "$\mathbf{\hat{\phi}}$", "Triangle"]
colors = ["C1", "C0", "C2", "C3"]

########## linneplots

ax = axes[0]
dts = ts_noisy.date.values
# for tt, label, color in zip(ts_arr, labels, colors):
ax.plot(dts, ts_truth, lw=3, label=labels[0], color=colors[0])
ax.plot(dts, ts_noisy, lw=1.5, label=labels[1], marker=".", color=colors[1])
ax.plot(dts, y_iter2_zeroed, lw=3, label=labels[2], color=colors[2])
ax.plot(dts, ts_tri_zeroed, lw=3, label=labels[3], color=colors[3])
# ax.plot(dts, ts_tri, lw=3)

ax.set_ylabel("[cm]")
ax.legend(loc="ur", ncols=2)

# ylim = ax.get_ylim() # Keep the same limits, but make a vert line for whole span
# ax.vlines(img_date, ylim[0], ylim[1], linestyle='--', color='gray', alpha=.5)
# ax.set_ylim(ylim)

#############################
# iidx = len(x) // 2
iidx = 53

zoom = (slice(120, 180), slice(120, 180))
# zoom = (slice(None), slice(None))

plotting.plot_img_diff(
    arrays=(
        dd_xr[iidx][zoom],
        dn_xr_lowess[iidx][zoom],
        dn_xr_tri[iidx][zoom],
    ),
    show_diff=False,
    titles=["$\mathbf{d}$", "LOWESS", "Triangle"],
    vm=3,
    cbar_label="[cm]",
    # cmap="RdBu_r",
    cmap="seismic_wide_y_r",
    axis_off=True,
    axes=axes[1:],
)

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure4-compare-tri.pdf")

In [ ]:
lowess._find_frac(x2, 430), 440 / 365

In [ ]:
# plt.figure()
# # plt.plot((dn_xr.data + day1noise_est[None, :, :])[:, 150, 150])
# plt.plot((dn_xr.data)[:, 150, 150])

In [ ]:
y = ts_noisy_spiked.values
y -= y[0]

# ts_tri_spike = signal.convolve(y, tri_50, mode='same') / sum(tri_50)
# ts_spike = lowess.lowess_pixel(y, x, frac=0.5)
ts_tri_spike = signal.convolve(y, tri_40, mode="same") / sum(tri_40)
ts_spike = lowess.lowess_pixel(y, x, frac=0.4)
# ts_tri_spike = signal.convolve(y, tri_30, mode='same') / sum(tri_30)
# ts_spike = lowess.lowess_pixel(y, x, frac=0.3)

ts_tri_spike -= ts_tri_spike[0]
ts_spike -= ts_spike[0]

In [ ]:
ts_arr = [ts_noisy_spiked, ts_truth, ts_tri_spike, ts_spike]
labels = ["Noisy", "Truth", "Triangle", "LOWESS"]


fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey=True, sharex=True)


for tt, label in zip(ts_arr, labels):
    ax.plot(ts_noisy.date, tt, lw=4, label=label, marker=".")


ax.set_title("Triangle vs. LOWESS with outliers")
ax.set_ylim((-2.5, 4))
ax.grid()
ax.legend(loc="lower right")

# Permian Basin

## Load data

In [ ]:
import cartopy.crs as ccrs
from apertools import sario
from cartopy.feature import ShapelyFeature

# import geopandas as gpd
from cartopy.io.shapereader import Reader

In [ ]:
datapath = "/Users/scott/Documents/Learning/masters-thesis/7year_merged_solutions_looked_18_bw15_2year_noref_all_el1/"

gis_dir = "/Users/scott/Documents/Learning/masters-thesis/fracking-qgis-data/"

data78 = Path(datapath + "los_path78")
data85 = Path(datapath + "los_path85")
data151 = Path(datapath + "los_path151")

data_merged = Path(datapath + "merged_cumulative_east_up")
data_clipped = Path(datapath + "clipped_cumulative_up")
data_clipped_diffs = Path(datapath + "clipped_diffs_up")

permian_clip_shape = gpd.read_file(datapath + "clip_shape_outline.geojson")
# permian_clip_shape.plot()
list(data78.glob("cumulative_los*"))[:2]

In [ ]:
# ds_78 = xr.open_dataset(datapath + "los_path78")
ds_85 = sario.load_xr_tifs(data85.glob("cumulative_los*"))
ds_151 = sario.load_xr_tifs(data151.glob("cumulative_los*"))

ds_up = sario.load_xr_tifs(data_merged.glob("*vertical*"))
ds_east = sario.load_xr_tifs(data_merged.glob("*east*"))

ds_up = sario.load_xr_tifs(data_merged.glob("*vertical*"))
ds_east = sario.load_xr_tifs(data_merged.glob("*east*"))

ds_clipped = sario.load_xr_tifs(data_clipped.glob("cumulative_vertical*"))
ds_clipped_1year = sario.load_xr_tifs((data_clipped_diffs / "1year").glob("*"))
ds_clipped_2year = sario.load_xr_tifs((data_clipped_diffs / "2year").glob("*"))

# change to end years
ds_clipped_1year["date"] = ds_clipped_1year.indexes["date"] + pd.DateOffset(years=1)
ds_clipped_2year["date"] = ds_clipped_2year.indexes["date"] + pd.DateOffset(years=2)

In [ ]:
permian_shape_path = (
    gis_dir + "repermianbasinshapefile/delaware_midland_central_basin.geojson"
)
gdf_permian = gpd.read_file(permian_shape_path).to_crs("EPSG:4326")
# fig, ax = pplt.subplots(projection='pcarree')
gdf_permian.plot(facecolor=(1, 1, 1, 0), edgecolor="purple", lw=2, alpha=0.3)

In [ ]:
ds_78

In [ ]:
pplt.rc["font.size"] = 12

In [ ]:
def plot_defo_image(
    ds,
    idx=-1,
    date=None,
    ax=None,
    vm=7,
    cmap="seismic_wide_y_r",
    lonticks=[-104, -103, -102],
    latticks=[31, 32],
    cbar_label="LOS [cm]",
    add_colorbar=True,
    bbox=None,
    plot_basin_shape=True,
    scale_bar_km=50,
    scale_loc=(0.8, 0.88),
    zebra_lw=3,
    
):
    # ds, idx = ds_78, -1
    if date is not None:
        img = ds.sel(date=date)
    else:
        img = ds[idx]
    crs = ccrs.PlateCarree()
    # crs = "pcarree"

    if ax is None:
        # fig, ax = pplt.subplots(proj=crs, figsize=(4, 4))#, sharex=True, sharey=True)
        fig = pplt.figure(
            refwidth=3,
            sharex=False,
            sharey=False,
        )
        ax = fig.subplot(
            abc="(a)",
            projection="pcarree",
            labels=False,
            dms=False,
        )
    else:
        fig = ax.figure

    if bbox is None:
        bbox = ds.rio.bounds()
    print(f"{bbox = }")
    # _, axim = plotting.map_img(
    #     img, bbox=bbox, pad_pct=0.0, ax=ax, vmax=vm, vmin=-vm, cmap=cmap
    # )
    _, ax = plotting.map_background(
        image=img,
        bbox=bbox,
        pad_pct=0.1,
        ax=ax,
        vmax=vm,
        vmin=-vm,
        cmap=cmap,
        interpolation="nearest",
    )
    axim = ax.get_images()[0]
    # axim = ax.imshow(np.random.rand(50, 50))
    if add_colorbar:
        ax.colorbar(axim, loc="r", label=cbar_label)

    if plot_basin_shape:
        gdf_permian.plot(
            alpha=0.3, facecolor="none", edgecolor="purple", lw=2, ax=ax, zorder=3
        )

    # ax.add_feature(shape_feature, zorder = 1)
    # perm = ax.add_feature(gdf['geometry'], crs=crs, alpha=.7, edgecolor="purple", lw=2, zorder=3)
    # perm.set_facecolor((1, 1, 0.5, 0.0))

    ax.format(
        # lonlim=(left, right),
        # latlim=(bot, top),
        labels=False,
        dms=False,
        lonlines=1,
        latlines=1,
    )
    ax.set_xticks(lonticks)
    ax.set_yticks(latticks)
    lw = 4
    ax.tick_params(axis="both", which="major", pad=lw)

    plotting.add_zebra(ax, lw=zebra_lw, zorder=3)

    plotting.scale_bar(ax, ax.projection, scale_bar_km, location=scale_loc, zorder=4)
    return fig, ax

In [ ]:
fig, ax = plot_defo_image(ds_78, idx=-1)

In [ ]:
# fig.savefig("~/test300.pdf", dpi=300)

In [ ]:
ds_up = xr.open_dataset(datapath + "los_path78")
ds_78 = sario.load_xr_tifs(data78.glob("cumulative_los*"))
ds_78

In [ ]:
ymin = ds_78.rio.bounds()[1]
ymax = ds_78.rio.bounds()[3]
ds_85 = ds_85.sel(lat=slice(ymax, ymin)).rio.set_spatial_dims("lon", "lat")
ds_151 = ds_151.sel(lat=slice(ymax, ymin)).rio.set_spatial_dims("lon", "lat")

In [ ]:
ds_78.rio.bounds(), ds_85.rio.bounds(), ds_151.rio.bounds()

## Plot 7 year cumulative for each path

In [ ]:
fig = pplt.figure(refwidth=2.5, refheight=1.8, sharex=False, sharey=False)
ax = fig.subplot(311, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(ds_151, idx=-1, ax=ax, lonticks=[-105, -104, -103])

ax = fig.subplot(312, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(ds_85, idx=-1, ax=ax, lonticks=[-105, -104, -103])

ax = fig.subplot(313, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(ds_78, idx=-1, ax=ax, lonticks=[-104, -103, -102])

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure-results-los.pdf", dpi=300)

## East up cumulative

In [ ]:
fig = pplt.figure(refwidth=3, refheight=2.3, sharex=False, sharey=False)
ax = fig.subplot(211, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(
    ds_up, idx=-1, ax=ax, lonticks=[-105, -104, -103], cmap="seismic_wide_y"
)

ax = fig.subplot(212, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(
    ds_east, idx=-1, ax=ax, lonticks=[-105, -104, -103], cmap="seismic_wide_y"
)

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure-results-east-up.pdf", dpi=300)

## Time series of interesting pixels

In [ ]:
lon, lat = -102.7712, 31.4970
zarr_path = "https://scott-insar.s3.us-west-004.backblazeb2.com/test-insar/defo_los_path{}.zarr/"
ds78full = xr.open_zarr(zarr_path.format(78))
# ds85full = xr.open_zarr(zarr_path.format(85))
# ds151full = xr.open_zarr(zarr_path.format(151))

In [ ]:
ds78full

In [ ]:
lon, lat = -102.7712, 31.4970
tsup = ds78full.sel(lon=lon, lat=lat, method="nearest")

In [ ]:
fig, ax = pplt.subplots()

tsup["defo_noisy"].plot(ax=ax, label="Noisy")
tsup["defo_lowess_2year"].plot(label="LOWESS")
ax.format(ylabel="LOS [cm]")
ax.legend(ncols=1, loc="lower left")

In [ ]:
lon, lat = -103.5582, 32.0468
bbox = -103.8, 31.9, -103.3, 32.3
left, bot, right, top = bbox
tsubs = ds78full.sel(lon=lon, lat=lat, method="nearest")
img_subs = ds78full["defo_lowess_2year"][-1].sel(
    lon=slice(left, right), lat=slice(top, bot)
)
img_subs.load()

In [ ]:
from shapely import geometry

fig, ax = pplt.subplots()
# ax.imshow(img_subs, cmap="RdBu")
img_subs.plot.imshow(cmap="RdBu", ax=ax)


box = gpd.GeoSeries(geometry.box(*bbox))

# print(circle)
circle = gpd.GeoSeries(geometry.Point(lon, lat).buffer(0.1))
circle.plot(ax=ax, facecolor="None", edgecolor="k")

In [ ]:
img_subs_window = utils.window_stack_xr(
    ds78full["defo_lowess_2year"].sel(date=slice(None, "2020-01-01")),
    lon=lon,
    lat=lat,
    window_size=9,
)

img_subs_window.plot()

In [ ]:
gdf_oil = sario.read_geopandas(gis_dir + "oil_production_2009-2019.csv")
gdf_water = sario.read_geopandas(gis_dir + "water_production_2009-2019.csv")
# gdf_water.columns, gdf_oil.columns

# https://stackoverflow.com/questions/9958506/element-wise-string-concatenation-in-numpy
years = np.arange(2015, 2019 + 1)
yearcols = np.char.add("y", years.astype(str))

gdf_oil = gdf_oil.loc[:, ["API", "geometry", "TopPerfDepth", *yearcols]]
gdf_water = gdf_water.loc[:, ["API", "geometry", "TopPerfDepth", *years.astype(str)]]

gdf_oil.rename(columns={"TopPerfDepth": "depth"}, inplace=True)
gdf_water.rename(columns={"TopPerfDepth": "depth"}, inplace=True)

gdf_oil = gdf_oil.rename({yc: yc.strip("y") for yc in yearcols}, axis=1)

gdf_oil.fillna(0.0, inplace=True)
gdf_water.fillna(0.0, inplace=True)

In [ ]:
gdf_water.head()

In [ ]:
# To get per-well totals
# agg_dict = {"volume": "sum", "geometry": "first", "depth": "first"}
# gdf_circle.groupby("API").agg(agg_dict)


def get_yearly_within_poly(gdf, poly):
    gdf2 = gdf.clip(poly).melt(
        id_vars=gdf.columns[:3],
        value_vars=gdf.columns[3:],
        var_name="year",
        value_name="volume",
    )
    return gdf2[["year", "volume"]].groupby("year").sum()

In [ ]:
gdf_water_circle = get_yearly_within_poly(gdf_water, circle)
gdf_oil_circle = get_yearly_within_poly(gdf_oil, circle)
gdf_oil_circle.head()

In [ ]:
gdf_circle = gdf_water_circle.join(gdf_oil_circle, lsuffix="_oil", rsuffix="_water")

In [ ]:
fig, axes = pplt.subplots(ncols=2, sharex=False, sharey=False, abc="(a)")

ax = axes[0]

# img_subs = img_subs.rio.set_spatial_dims('lon', 'lat')
# axim = ax.imshow(img_subs, cmap="RdBu", extent=img_subs.rio.bounds())

axim = img_subs.plot.imshow(ax=ax, cmap="RdBu", add_colorbar=False)
ax.colorbar(axim, loc="r", label="LOS [cm]")
ax.format(xlabel="", ylabel="", title="")

circle.plot(ax=ax, facecolor="None", edgecolor="k")

gdf_water["geometry"].clip(box).plot(ax=ax, ms=2, edgecolor="none", facecolor="gray")


###
ax = axes[1]
ax.bar(gdf_circle.index, gdf_circle.values / 1e6, stack=True, labels=["Water", "Oil"])
ax.format(ylabel="Production [MBBl]", xlabel="")
ax.legend(ncols=1)

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure-los-path78-production.pdf", dpi=300)

## Get injection timeseries

In [ ]:
gdf = sario.read_geopandas(
    "/Users/scott/Documents/Learning/masters-thesis/fracking-qgis-data/water_injection_2009-2019.csv"
)
years = np.arange(2015, 2019 + 1)
gdf = gdf[gdf["Type"] == "SWD"]  # only salt water disposal, not EOR
gdf = gdf.loc[:, ["API", "geometry", "TopPerfDepth", *years.astype(str)]]

gdf.rename(columns={"TopPerfDepth": "depth"}, inplace=True)
gdf.fillna(0.0, inplace=True)
gdf

In [ ]:
gdf2 = gdf.melt(
    id_vars=gdf.columns[:3],
    value_vars=gdf.columns[3:],
    var_name="year",
    value_name="volume",
)

gdf2["year"] = pd.to_datetime(gdf2["year"], format="%Y")

gdf2

In [ ]:
kim_lu_api = 4249533675
gdf2_inj = gdf2[gdf2["API"] == kim_lu_api]
gdf2_inj

In [ ]:
lon, lat = gdf2_inj.iloc[0].geometry.coords[0]
tsinj = ds78full.sel(lon=lon, lat=lat, method="nearest")

In [ ]:
gdf2_inj

In [ ]:
(tsinj["defo_noisy"].indexes["date"] - tsinj["defo_noisy"].indexes["date"][0])[:25]

In [ ]:
mode = "reflect"
# about 2 years
tri_win = signal.windows.triang(25)

dn_tri = ndimage.convolve1d(tsinj["defo_noisy"].data, tri_win, axis=0, mode=mode) / sum(
    tri_win
)
dn_tri = dn_tri - dn_tri[0]
dn_xr_tri = tsinj["defo_noisy"].copy()
dn_xr_tri.data = dn_tri
# dn_xr_tri

In [ ]:
tsinj.load();

In [ ]:
fig, ax = pplt.subplots(refwidth=3.5, refheight=2.8)

# ds = pd.to_datetime(tsinj.indexes['date']).to_series()
ds = tsinj.indexes["date"].to_pydatetime()

# tsinj['defo_noisy'].plot(ax=ax, label='Noisy')
# tsinj['defo_lowess_2year'].plot(ax=ax, label='LOWESS')
l1 = ax.plot(ds, tsinj["defo_noisy"].values, label="Noisy")
l2 = ax.plot(ds, tsinj["defo_lowess_2year"].values, label="LOWESS", lw=3)
# tsinj['defo_lowess_2year'].plot(ax=ax, label='LOWESS')
# l3 = ax.plot(ds, dn_xr_tri.values, label='Tri', lw=3, color='C3')

ax.format(ylabel="LOS [cm]", grid=True)


ax2 = ax.twinx()
ax2.set_zorder(1)
ax.set_zorder(2)
# gdf2_inj[['year', 'volume']].plot(kind='bar', x='year', ax=ax)
# gdf2_inj[['year', 'volume']].set_index('year').plot(ax=ax2, marker='o')
# li = ax2.plot(gdf2_inj['year'].values, gdf2_inj['volume'] / 1e6, marker='o', color='C2')
li = ax2.bar(
    gdf2_inj.year.dt.to_pydatetime(), gdf2_inj["volume"] / 1e6, color="C2", alpha=0.6
)
ax2.format(ylabel="Yearly Injection Volume (MBBl)")


# ax.legend([l1, l2, l3, li], ['Noisy', 'LOWESS', 'Triangle', 'Injection'], ncols=1, loc='lower left')
ax.legend([l1, l2, li], ["Noisy", "LOWESS", "Injection"], ncols=1, loc="lower right")

In [ ]:
(dn_xr_tri - tsinj["defo_lowess_2year"]).plot()

## Path 78 LOS and injections

In [ ]:
import matplotlib.patches as patches


def plot_rect(
    ax,
    lon,
    lat,
    w=0.05,
    edgecolor="k",
    lw=3,
    zorder=5,
):
    xy = (lon - w / 2, lat - w / 2)
    rect = patches.Rectangle(
        xy, w, w, facecolor="none", edgecolor=edgecolor, lw=lw, zorder=zorder
    )
    return ax.add_patch(rect)

In [ ]:
fig = pplt.figure(refwidth=3, sharex=False, sharey=False)

ax = fig.subplot(121, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(ds_78, idx=-1, ax=ax, lonticks=[-104, -103, -102])
# Add rect on time series area
w = 0.15
plot_rect(ax, lon, lat, w=w, zorder=5)

# Injection timeseries
ax = fig.subplot(122, abc="(a)")
ds = tsinj.indexes["date"].to_pydatetime()

l1 = ax.plot(ds, tsinj["defo_noisy"].values, label="Noisy")
l2 = ax.plot(ds, tsinj["defo_lowess_2year"].values, label="LOWESS", lw=3)
# l3 = ax.plot(ds, dn_xr_tri.values, label='Tri', lw=3, color='C3')

ax.format(ylabel="LOS [cm]", grid=True)


ax2 = ax.twinx()
ax2.set_zorder(1)
ax.set_zorder(2)
li = ax2.bar(
    gdf2_inj.year.dt.to_pydatetime(), gdf2_inj["volume"] / 1e6, color="C2", alpha=0.6
)
ax2.format(ylabel="Yearly Injection Volume (MBBl)")

# ax.legend([l1, l2, l3, li], ['Noisy', 'LOWESS', 'Triangle', 'Injection'], ncols=1, loc='lower left')
ax.legend([l1, l2, li], ["Noisy", "LOWESS", "Injection"], ncols=1, loc="lower left")

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure-results-los-path78.pdf", dpi=300)

In [ ]:
!pwd

## Earthquakes and diff images

In [ ]:
gdf_eqs = sario.read_geopandas(
    gis_dir + "texnet_events_20220414.csv",
    latcol="Latitude (WGS84)",
    loncol="Longitude (WGS84)",
)
# gdf_eqs_growclust = sario.read_geopandas(gis_dir + "texnet_growclust_20211208.csv", latcol='latC', loncol='lonC')
gdf_eqs_growclust = sario.read_geopandas(
    gis_dir + "texnet_growclust_20211208.csv", latcol="latR", loncol="lonR"
)

In [ ]:
# gdf_eqs_growclust.plot(markersize=1)

In [ ]:
ds_clipped_2year.date

In [ ]:
(pd.to_datetime(d1) + pd.DateOffset(years=2)).strftime("%Y%m%d")

In [ ]:
def dt_to_title(d, span=2):
    start = pd.to_datetime(d) - pd.DateOffset(years=span) + pd.DateOffset(days=2)
    # end = pd.to_datetime(d) + pd.DateOffset(years=span)
    end = pd.to_datetime(d)
    return f"{start.strftime('%Y')} - {end.strftime('%Y')}"

dt_to_title(d1)

In [ ]:
def dt_to_title(d, span=2):
    start = pd.to_datetime(d) - pd.DateOffset(years=span) + pd.DateOffset(days=2)
    end = pd.to_datetime(d) # + pd.DateOffset(years=span)
    return f"{start.strftime('%Y')} - {end.strftime('%Y')}"


# End dates for 2 year diffs
d1, d2, d3 = "2017-12-31", "2019-12-31", "2021-12-31"
cmap = "seismic_wide_y"
vm = 2.5
lonticks = [-104.5, -103.5]
# bbox = ds_clipped_2year.rio.bounds()
bbox = (-104.7, 30.8, -103, 32.5)

fig = pplt.figure(refwidth=2., refheight=1.8, sharex=False, sharey=False)

for n, dd in zip([321, 323, 325], [d1, d2, d3]):
    ax = fig.subplot(n, abc="(a)", projection="pcarree", labels=False, dms=False)
    plot_defo_image(ds_clipped_2year, date=dd, ax=ax, lonticks=lonticks, cmap=cmap, vm=vm, bbox=bbox)
    ax.set_title(dt_to_title(dd))


## With earthquakes
for n, dd, yrs in zip(
    [322, 324, 326], [d1, d2, d3], [[2017], [2018, 2019], [2020, 2021]]
):
    ax = fig.subplot(n, abc="(a)", projection="pcarree", labels=False, dms=False)
    plot_defo_image(
        ds_clipped_2year, date=dd, ax=ax, lonticks=lonticks, cmap=cmap, vm=vm, bbox=bbox, add_colorbar=False,
    )
    gdf_eqs_growclust[gdf_eqs_growclust.yr.isin(yrs)].plot(
        markersize=10, ax=ax, zorder=5, column="depR", cmap='Purples', legend=True,
        vmax=7, vmin=2.5, legend_kwds={'label':"Depth [km]"},
    )
    ax.set_title(dt_to_title(dd))

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure-results-eqs.pdf", dpi=300)

In [ ]:
gdf_eqs_growclust.columns

In [ ]:
# bbox = ds_clipped_2year.rio.bounds()
bbox = (-104.7, 30.8, -103, 32.8)

# def plot_eqs(gdf,  
crs = ccrs.PlateCarree()
fig = pplt.figure(
    refwidth=3,
    sharex=False,
    sharey=False,
)
ax = fig.subplot(
    abc="(a)",
    projection="pcarree",
    labels=False,
    dms=False,
)

_, ax = plotting.map_background(
    bbox=bbox,
    pad_pct=0.1,
    ax=ax,
)
gdf_eqs_growclust[gdf_eqs_growclust.yr.isin([2017])].plot(
    markersize=10, ax=ax, zorder=5, column="depR", cmap='Purples', legend=True, legend_kwds={'label':"Depth [km]"},
)

In [ ]:
bbox = (-104.62, 31.45, -103.9, 31.9)
yrs = [2020, 2021]
dd = d3
lonticks = [-104.5, -104.25, -104.0]
latticks = [31.5, 31.75]
scale_bar_km = 20
scale_loc = (0.2, 0.88)

fig = pplt.figure(refwidth=3.3, refheight=2, sharex=False, sharey=False)

ax = fig.subplot(211, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(
    ds_clipped_2year, date=dd, ax=ax, lonticks=lonticks, latticks=latticks, 
    cmap=cmap, vm=vm, bbox=bbox, scale_bar_km=scale_bar_km, scale_loc=scale_loc,
)
ax.set_title(dt_to_title(dd))


ax = fig.subplot(212, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(
    ds_clipped_2year, date=dd, ax=ax, lonticks=lonticks, latticks=latticks,
    cmap=cmap, vm=vm, bbox=bbox,  scale_bar_km=scale_bar_km, scale_loc=scale_loc,
    add_colorbar=False,
)
gdf_eqs_growclust[gdf_eqs_growclust.yr.isin(yrs)].plot(
    markersize=10, ax=ax, zorder=5, column="depR", cmap='Purples', legend=True, legend_kwds={'label':"Depth [km]"},
    vmax=7, vmin=2.5
)
ax.set_title(dt_to_title(dd))

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure-cmez-zoom.pdf", dpi=300)

In [ ]:
2.7/1.6 * 2

In [ ]:
from apertools import latlon
npoints = 500

# start1 = -104.22, 31.8
# end1 = -104.49, 31.475
start1 = -104.19, 31.75
end1 = -104.15, 31.49

transect1 = utils.transect(ds_clipped_2year, start1, end1, npoints)
total_dist1 = latlon.latlon_to_dist(start1[::-1], end1[::-1])
transect1['index'] = transect1['index'] / transect1['index'].max() * total_dist1 / 1e3


start2 = -104.368, 31.75
end2 = -104.4, 31.55
transect2 = utils.transect(ds_clipped_2year, start2, end2, npoints)
total_dist2 = latlon.latlon_to_dist(start2[::-1], end2[::-1])
transect2['index'] = transect2['index'] / transect2['index'].max() * total_dist2 / 1e3


transect_sm1 = transect1.rolling(index=10, min_periods=1).mean()[::2] # get only 1/3/5th transect
transect_sm2 = transect2.rolling(index=10, min_periods=1).mean()[::2]

transect_sm1

In [ ]:
labels = [dt_to_title(dd) for dd in [d1, d2, d3]]
# fig, axes = pplt.subplots(ncols=2, refwidth=3, share=False)
fig = pplt.figure(refwidth=3.3, refheight=2, sharex=False, sharey=False)


ax = fig.subplot(121, abc='(a)')
# ax = axes[0]
lines = transect_sm1.plot.line(hue='date', add_legend=False, ax=ax)
ax.legend(lines, labels, ncols=1)
ax.format(ylabel="[cm]")

# ax = axes[1]
ax = fig.subplot(122, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(
    ds_clipped_2year, date=dd, ax=ax, lonticks=lonticks, latticks=latticks, 
    cmap=cmap, vm=vm, bbox=bbox, scale_bar_km=scale_bar_km, scale_loc=scale_loc,
)
ax.plot(transect_sm1.lon, transect_sm1.lat, zorder=3, lw=2, linestyle='dashed')
ax.plot(transect_sm2.lon, transect_sm2.lat, zorder=3, lw=2, linestyle='dashed')


In [ ]:
bbox = (-104.62, 31.45, -103.9, 31.9)
yrs = [2020, 2021]
dd = d3
lonticks = [-104.5, -104.25, -104.0]
latticks = [31.5, 31.75]
scale_bar_km = 20
scale_loc = (0.2, 0.88)


fig = pplt.figure(refwidth=3.3, refheight=2, sharex=False, sharey=False)

ax = fig.subplot(211, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(
    ds_clipped_2year, date=dd, ax=ax, lonticks=lonticks, latticks=latticks, 
    cmap=cmap, vm=vm, bbox=bbox, scale_bar_km=scale_bar_km, scale_loc=scale_loc,
)
ax.set_title(dt_to_title(dd))
ax.plot(transect_sm1.lon, transect_sm1.lat, zorder=3, lw=2, linestyle='dashed')
ax.plot(transect_sm2.lon, transect_sm2.lat, zorder=3, lw=2, linestyle='dashed')



ax = fig.subplot(212, abc="(a)", projection="pcarree", labels=False, dms=False)
plot_defo_image(
    ds_clipped_2year, date=dd, ax=ax, lonticks=lonticks, latticks=latticks,
    cmap=cmap, vm=vm, bbox=bbox,  scale_bar_km=scale_bar_km, scale_loc=scale_loc,
    add_colorbar=False,
)
gdf_eqs_growclust[gdf_eqs_growclust.yr.isin(yrs)].plot(
    markersize=10, ax=ax, zorder=5, column="depR", cmap='Purples', legend=True, legend_kwds={'label':"Depth [km]"},
    vmax=7, vmin=2.5
)
ax.set_title(dt_to_title(dd))

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure-cmez-zoom-transect.pdf", dpi=300)

In [ ]:
labels = [dt_to_title(dd) for dd in [d1, d2, d3]]
fig, axes = pplt.subplots(ncols=2, refwidth=3, refheight=2.4, share=False, abc='(a)')

ax = axes[0]
lines = transect_sm1.plot.line(hue='date', add_legend=False, ax=ax)
ax.legend(lines, labels, ncols=1, loc='ur')
ax.format(ylabel="[cm]", title="", xlabel="Distance along transect [km]")

ax = axes[1]
lines = transect_sm2.plot.line(hue='date', add_legend=False, ax=ax)
ax.legend(lines, labels, ncols=1, loc='ur')
ax.format(ylabel="[cm]", title="", xlabel="Distance along transect [km]")


In [ ]:
fig.savefig("../figures/chapter5-lowess/figure-cmez-transects.pdf", dpi=300)